# 🌐 Proyecto Integrador Senior 2: Data Mesh Multi-Dominio con Feature Store

Objetivo: diseñar una arquitectura Data Mesh con múltiples dominios autónomos, feature store centralizado para ML, y gobernanza federada.

- Duración: 180+ min (proyecto multi-día)
- Dificultad: Muy Alta
- Prerrequisitos: Senior completo (01–08), experiencia organizacional

## 1. Contexto y requerimientos

**Empresa**: Marketplace multi-categoría con 5 dominios de negocio:
- Ventas (Sales)
- Logística (Fulfillment)
- Producto (Catalog)
- Marketing (Campaigns)
- Finanzas (Payments)

**Objetivo**: Cada dominio gestiona sus propios datos como producto, con SLOs, versionado y documentación. Un feature store central consume features de todos los dominios para ML.

**Requerimientos**:
- Plataforma self-service: catálogo, CI/CD, observabilidad compartidos.
- Gobernanza federada: políticas de seguridad y calidad globales, aplicadas localmente.
- Feature store (Feast/Tecton) con features de cada dominio.
- APIs de data products con contratos versionados (OpenAPI).
- Linaje cross-domain visible en DataHub.

## 2. Arquitectura Data Mesh propuesta

In [ ]:
mesh_diagram = '''
┌────────────────────────────────────────────────────────────────┐
│                  Plataforma Self-Service                       │
│  - Airflow compartido  - DataHub (catálogo + linaje)          │
│  - Grafana + Prometheus - CI/CD (GitHub Actions)              │
│  - Feature Store (Feast) - Políticas IAM centrales            │
└────────────────────────────────────────────────────────────────┘
                              ▲
          ┌───────────────────┼───────────────────┐
          │                   │                   │
  ┌───────▼──────┐   ┌────────▼────────┐  ┌──────▼────────┐
  │ Dominio      │   │  Dominio        │  │  Dominio      │
  │ Ventas       │   │  Logística      │  │  Producto     │
  │ (data prod)  │   │  (data prod)    │  │  (data prod)  │
  │ - raw/       │   │  - raw/         │  │  - raw/       │
  │ - curated/   │   │  - curated/     │  │  - curated/   │
  │ - API        │   │  - API          │  │  - API        │
  │ - Features   │   │  - Features     │  │  - Features   │
  └──────────────┘   └─────────────────┘  └───────────────┘
          │                   │                   │
          └───────────────────┼───────────────────┘
                              ▼
                   ┌──────────────────┐
                   │  Feature Store   │
                   │  (Feast/Tecton)  │
                   └─────────┬────────┘
                             │
                      ┌──────▼─────┐
                      │ ML Models  │
                      │ (training) │
                      └────────────┘
'''
print(mesh_diagram)

## 3. Componentes por dominio (ejemplo: Ventas)

### 3.1 Data Product Ventas
- Owner: Equipo de Ventas.
- Fuentes: Kafka (transacciones), archivos batch (devoluciones).
- Storage: S3 `s3://mesh/ventas/raw/`, `s3://mesh/ventas/curated/`.
- API: FastAPI endpoint `/ventas/v1/daily-revenue` con contrato OpenAPI.
- Features: `cliente_total_compras_30d`, `cliente_num_transacciones_7d`.
- SLO: latencia p99 < 15 min, disponibilidad > 99.9%.
- Documentación: README, diagramas, changelog.

### 3.2 Pipeline Ventas
- Airflow DAG propio del equipo, con validaciones GE y alertas.
- Escribe features a Feast (offline store: S3 Parquet, online: Redis).
- Emite linaje a DataHub vía OpenLineage.

## 4. Feature Store centralizado

In [ ]:
feast_config = r'''
# feature_repo/ventas_features.py
from feast import Entity, FeatureView, Field, FileSource
from feast.types import Float32, Int64
from datetime import timedelta

cliente = Entity(name='cliente_id', join_keys=['cliente_id'])

ventas_source = FileSource(
    path='s3://mesh/ventas/curated/features.parquet',
    timestamp_field='event_timestamp'
)

ventas_fv = FeatureView(
    name='ventas_features',
    entities=[cliente],
    ttl=timedelta(days=30),
    schema=[
        Field(name='total_compras_30d', dtype=Float32),
        Field(name='num_transacciones_7d', dtype=Int64),
    ],
    source=ventas_source,
    owner='ventas-team@empresa.com'
)

# Similarmente para logistica_features, producto_features, etc.
'''
print(feast_config.splitlines()[:25])

## 5. Gobernanza federada

- Políticas globales:
  - Todo PII enmascarado en datasets compartidos.
  - Validaciones mínimas de calidad (Great Expectations).
  - Linaje obligatorio (OpenLineage).
  - Versionado semántico de APIs.
- Autonomía local:
  - Cada dominio elige su stack de transformación (Spark/Pandas/dbt).
  - Frecuencia de actualizaciones según SLO propio.
  - Esquemas propios, evolucionables con compatibilidad (Avro/Protobuf).

## 6. Checklist de implementación

In [ ]:
checklist = '''
☐ 1. Definir dominios y owners (RACI)
☐ 2. Crear buckets S3 por dominio (ventas/, logistica/, producto/)
☐ 3. DAG Airflow por dominio con validaciones y linaje
☐ 4. APIs FastAPI versionadas (OpenAPI specs)
☐ 5. Feature definitions en Feast por dominio
☐ 6. Feast apply y materialize-incremental en CI/CD
☐ 7. DataHub registrar data products con metadata
☐ 8. Políticas IAM federadas (admin global + roles por dominio)
☐ 9. Dashboard Grafana multi-dominio con SLOs
☐ 10. Contratos de calidad (data contracts) versionados
☐ 11. Onboarding docs para nuevos dominios
☐ 12. Incident response playbook cross-domain
☐ 13. Cost allocation tags por dominio
☐ 14. Training model multi-dominio (consume features de Feast)
☐ 15. Tests de integración cross-domain
'''
print(checklist)

## 7. Entregables

- Documento de diseño Data Mesh con principios y responsabilidades.
- Repositorio multi-dominio (monorepo o multi-repo).
- Feature store funcional con features de ≥3 dominios.
- Catálogo DataHub con linaje cross-domain.
- APIs documentadas (Swagger/OpenAPI) por dominio.
- Dashboard unificado con métricas de todos los dominios.
- Modelo ML entrenado consumiendo features federadas.
- Presentación ejecutiva (slides) con resultados y aprendizajes.

## 8. Evaluación

- Autonomía: ¿cada dominio opera independiente?
- Gobernanza: ¿políticas aplicadas consistentemente?
- Feature store: ¿features accesibles y versionadas?
- Observabilidad: ¿linaje y métricas cross-domain?
- Escalabilidad: ¿fácil agregar nuevos dominios?
- Costos: ¿optimización por dominio visible?